## Repeat from Alpaca 1

In [22]:
import pandas as pd
import numpy as np
from datetime import datetime
from alpaca.trading.client import TradingClient
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockLatestQuoteRequest
from alpaca.trading.requests import MarketOrderRequest, GetAssetsRequest
from alpaca.trading.enums import OrderSide, TimeInForce, AssetClass

KEY = "PKN99TWE381RXZFC5P04"
SECRET_KEY = "e6kwHgPJoGr9sanjK2uTmqTynQfg7tvQXBMgR6lW"

data_client = StockHistoricalDataClient(KEY, SECRET_KEY)
trading_client = TradingClient(KEY, SECRET_KEY, paper=True)

today = datetime.today().strftime("%Y-%m-%d")

numstocks = 200

#### Get stock predictions

In [23]:
df = pd.read_csv(f"files/predictions_{today}.csv", index_col="ticker")
ticks = df.index.to_list()

#### Get account equity

In [24]:
account = trading_client.get_account()
equity = float(account.equity)

long_per_stock = 1.3*equity / numstocks
short_per_stock = 0.3*equity / numstocks

#### Get tradeable and shortable stocks

In [25]:
assets = trading_client.get_all_assets()

assets = [x for x in assets if (x.asset_class[:]=='us_equity') and (x.symbol in ticks) and (x.status[:]=='active')]
symbols = [x.symbol for x in assets]
tradable = [x.tradable for x in assets]
shortable = [x.shortable for x in assets]

tradable = pd.Series(tradable, index=symbols)
shortable = pd.Series(shortable, index=symbols)


df["tradable"] = tradable
df["shortable"] = shortable

df = df[~df.tradable.isna() & df.tradable]
ticks = df.index.to_list()

#### Get quotes and quantities

In [26]:

params = StockLatestQuoteRequest(symbol_or_symbols=ticks)
quotes = data_client.get_stock_latest_quote(params)
df["ask"] = [quotes[x].ask_price for x in ticks]
df["bid"] = [quotes[x].bid_price for x in ticks]

qlong = [int(long_per_stock/x) if x>0 else np.nan for x in df.ask]
qshort = [-int(short_per_stock/x) if x>0 else np.nan for x in df.bid]

#### Calculate target positions

In [27]:
df["rnk"] = df.predict.rank(method="first")
df["portfolio"] = [
    "worst" if x<=numstocks else ("best" if x>df.shape[0]-numstocks else "middle") for x in df.rnk
]
df["qty"] = np.where(df.portfolio=="worst", qshort, 0)
df["qty"] = np.where(df.portfolio=="best", qlong, df.qty)

## New: Trade to Update Positions

#### Get positions

In [29]:
positions = trading_client.get_all_positions()
positions = {x.symbol: int(x.qty) for x in positions}
positions = pd.Series(positions)
df["current"] = positions
df["current"] = df.current.fillna(0)

#### A simple protocol: trade to target quentities

In [30]:
df["qty"] = df.qty - df.current

## Same Order Submission as in Alpaca 1

In [67]:

for tick in df.index: 
    if df.loc[tick, "qty"]<0:
        try:
            market_order_data = MarketOrderRequest(
                symbol=tick,
                qty=-df.loc[tick, "qty"],
                side=OrderSide.SELL,
                time_in_force=TimeInForce.DAY
            )
            market_order = trading_client.submit_order(
                order_data=market_order_data
            )
        except:
            print(f"sell order for {tick} failed")
    elif df.loc[tick, "qty"]>0:
        try:
            market_order_data = MarketOrderRequest(
                symbol=tick,
                qty=df.loc[tick, "qty"],
                side=OrderSide.BUY,
                time_in_force=TimeInForce.DAY
            )
            market_order = trading_client.submit_order(
                order_data=market_order_data
            )
        except:
            print(f"buy order for {tick} failed")

